In [116]:
# !brew install hbase
# !pip3 install thrift
# !pip3 install hbase-thrift
# !pip3 install happybase
# !start-hbase.sh
# !hbase thrift start
# table: certificate
# column family: feature

In [9]:
import happybase
import requests
import pandas as pd
import numpy as np

stock_code = '601116'
licence = '9831453184924e2ed3'
table_name = 'certificate'
column_family = 'feature'

In [10]:
# connection = happybase.Connection('localhost', 9090, transport='framed', protocol='compact')
# connection.open()
# connection.create_table(name=table_name,families={column_family:dict()})
# print(connection.tables())
# connection.close()

In [15]:
def dump_history_specific_data(code, url, key_col, feature_type):
    response = requests.get(url=url)
    history = response.json()
    connection = happybase.Connection('localhost', 9090, transport='framed', protocol='compact')
    connection.open()
    table = connection.table(name=table_name)
    print(history)
    for item in history:
        rowkey = code + '_' + item[key_col]
        for col_name in item:
            if col_name != key_col:
                column_qulifier = 'feature:' + feature_type + '_' + col_name
                value = str(item[col_name])
                table.put(row=rowkey, data={column_qulifier:value})
    connection.close()

def dump_history_data(code, licence):
    #dump_history_specific_data(code=code, url='http://api.mairui.club/zs/hfsjy/{}/dn/{}'.format(code, licence), key_col='d', feature_type='trade')
    #dump_history_specific_data(code=code, url='http://api.mairui.club/zs/hkdj/{}/dn/{}'.format(code, licence), key_col='t', feature_type='kdj')
    #dump_history_specific_data(code=code, url='http://api.mairui.club/zs/hmacd/{}/dn/{}'.format(code, licence), key_col='t', feature_type='macd')
    pass

dump_history_data(code=stock_code, licence=licence)

[{'t': '2011-03-14', 'k': '9.6300', 'd': '9.6300', 'j': '9.63'}, {'t': '2011-03-15', 'k': '8.71', 'd': '9.32', 'j': '7.49'}, {'t': '2011-03-16', 'k': '12.18', 'd': '10.27', 'j': '16.00'}, {'t': '2011-03-17', 'k': '10.50', 'd': '10.35', 'j': '10.80'}, {'t': '2011-03-18', 'k': '10.37', 'd': '10.36', 'j': '10.39'}, {'t': '2011-03-21', 'k': '8.85', 'd': '9.86', 'j': '6.83'}, {'t': '2011-03-22', 'k': '9.55', 'd': '9.76', 'j': '9.13'}, {'t': '2011-03-23', 'k': '14.99', 'd': '11.50', 'j': '21.97'}, {'t': '2011-03-24', 'k': '17.55', 'd': '13.51', 'j': '25.63'}, {'t': '2011-03-25', 'k': '21.14', 'd': '16.05', 'j': '31.32'}, {'t': '2011-03-28', 'k': '21.82', 'd': '17.97', 'j': '29.52'}, {'t': '2011-03-29', 'k': '15.07', 'd': '17.00', 'j': '11.21'}, {'t': '2011-03-30', 'k': '11.54', 'd': '15.18', 'j': '4.26'}, {'t': '2011-03-31', 'k': '30.94', 'd': '20.43', 'j': '51.96'}, {'t': '2011-04-01', 'k': '43.66', 'd': '28.17', 'j': '74.64'}, {'t': '2011-04-06', 'k': '40.74', 'd': '32.36', 'j': '57.50'}, 

In [16]:
connection = happybase.Connection('localhost', 9090, transport='framed', protocol='compact')
connection.open()
table = connection.table(table_name)
rows = table.scan(row_prefix=stock_code.encode('utf-8'))
df = pd.DataFrame()
idx = int(0)
for row in rows:
    date = row[0].decode('utf-8').split('_')[-1]
    cells = row[1]
    for column in cells:
        df.at[idx, column.decode('utf-8').split(':')[-1]] = np.float32(cells[column].decode('utf-8'))
    df.at[idx, 'date'] = date
    idx = idx + 1
connection.close()
df.dropna(inplace=True)
df.set_index('date')
df.sort_index(inplace=True)
df

,macd_dea,macd_diff,macd_ema12,macd_ema26,macd_macd,trade_c,trade_e,trade_h,trade_hs,trade_l,trade_o,trade_v,trade_zd,trade_zde,trade_zf,date,kdj_d,kdj_j,kdj_k
8,-0.670,-0.982,19.244801,20.2272,-0.625,18.110001,131154608.0,18.440001,15.07,17.959999,18.219999,72348.0,-2.11,-0.39,2.59,2011-03-14,9.630000,9.630000,9.630000
9,-0.739,-1.018,19.036400,20.0541,-0.557,17.889999,93989448.0,18.299999,10.93,17.660000,18.110001,52441.0,-1.21,-0.22,3.53,2011-03-15,9.320000,7.490000,8.710000
10,-0.798,-1.031,18.864599,19.8960,-0.467,17.920000,72221864.0,18.010000,8.41,17.799999,17.879999,40356.0,0.17,0.03,1.17,2011-03-16,10.270000,16.000000,12.180000
11,-0.856,-1.086,18.611601,19.6978,-0.461,17.219999,112627296.0,17.799999,13.52,17.100000,17.799999,64905.0,-3.91,-0.70,3.91,2011-03-17,10.350000,10.800000,10.500000
12,-0.907,-1.113,18.405199,19.5180,-0.412,17.270000,44160112.0,17.440001,5.33,17.139999,17.250000,25577.0,0.29,0.05,1.74,2011-03-18,10.360000,10.390000,10.370000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2989,0.102,0.161,12.282600,12.1213,0.119,12.440000,217834208.0,12.750000,3.20,12.230000,12.470000,175075.0,-0.16,-0.02,4.17,2023-07-14,49.090000,43.119999,47.099998
2990,0.114,0.164,12.309900,12.1464,0.099,12.460000,130866688.0,12.460000,1.94,12.130000,12.340000,106216.0,0.16,0.02,2.65,2023-07-17,47.770000,39.849998,45.130001
2991,0.134,0.213,12.428400,12.2156,0.158,13.080000,420868640.0,13.200000,5.98,12.320000,12.410000,327400.0,4.98,0.62,7.06,2023-07-18,49.340000,58.790001,52.490002
2992,0.157,0.249,12.528600,12.2796,0.184,13.080000,572291584.0,14.390000,7.74,12.810000,12.830000,424071.0,0.00,0.00,12.08,2023-07-19,50.259998,55.750000,52.090000


In [193]:
connection = happybase.Connection('localhost', 9090, transport='framed', protocol='compact')
connection.open()
table = connection.table(table_name)
rows = table.scan(row_prefix=stock_code.encode('utf-8'))
for row in rows:
    print(row)
connection.close()
